In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import math
import numpy as np
from numpy import random
from math import pi
import matplotlib.pyplot as plt
from math import sqrt
from  sklearn import mixture
import scipy.sparse
from scipy.stats import norm
from scipy.stats import multivariate_normal
tfd = tfp.distributions
from sklearn.utils import shuffle


In [3]:
def pm0(x, m, s):
    mvn = tfd.MultivariateNormalFullCovariance(
    loc=m,
    covariance_matrix=s)
    k = len(m)
    return np.sqrt(np.linalg.det(s)*(2*pi)**k)*mvn.prob(x);


def pn(x, m0, s0):
    mvn = tfd.MultivariateNormalFullCovariance(
    loc=m0,
    covariance_matrix=s0)
    
    return mvn.prob(x);

In [4]:
class Gradient:


    # instance attribute
    def __init__(self, cte,mu,sigma, error_mu,error_sigma, error_cte, ctes,mus,sigmas):
        self.cte = cte
        self.mu = mu
        self.sigma = sigma
        
        self.error_mu = error_mu
        self.error_sigma = error_sigma
        self.error_cte = error_cte 
        
        self.ctes = ctes
        self.mus = mus
        self.sigmas = sigmas
        

In [5]:
def obj_f2():
    
    g1= a*Z1 + b*Z2 + mu[0]
    g2 = c*Z1 +  d*Z2 + mu[1]

    

    g= tf.stack([g1,g2],1)


    p_n = pn( g,m0, S) # noise (generator) density 
    p_m = pm0(g, m_data, s_data) #data density
    return tf.math.reduce_mean( tf.math.log(p_n/(cte*p_m+p_n)))

In [ ]:

######## ALTERNATIVE VERSION #############

# d =2
m_data = [1,2]
s_data = [[2.0, 0.5], [0.5, 3]]

batch_size=100
x_batches = multivariate_normal.rvs(m_data, s_data, 10,batch_size)


mu_init = [1, 2]
a,b,c,d =  tf.Variable((1+1.41421356), dtype="float32"), tf.Variable(1+0, dtype="float32"), tf.Variable(1+0.35355339, dtype="float32"), tf.Variable(1+1.6955825 ,dtype="float32")
cte_init = 1

mu = tf.Variable(mu_init, dtype="float32")

cte = cte_init
error_mu = [] 
error_sigma = []


error_cte = [] 

mus = []
sigmas = []


ctes = []
max_iters = 500
learning_rate = 0.01
opt = tf.keras.optimizers.SGD(learning_rate=0.01)
 
for itr in range(max_iters): 
    #print(itr)
    for x in x_batches:
        ctes.append(cte)
        Z1 = np.array(random.normal( 0, 1,batch_size)).astype("float32")
        Z2 = np.array(random.normal( 0, 1,batch_size)).astype("float32")
        
        g1= a*Z1 + b*Z2 + mu[0]
        g2 = c*Z1 +  d*Z2 + mu[1]

        A = np.array([[a,b],[c,d]])
        S = np.array(np.matmul(A , np.transpose(A))).astype("float32")
        
       

        g= np.stack([g1,g2],1)
        
       
        
        m0 = mu.numpy()
        
        p_nx = pn( x,m0, S).numpy() # noise (generator) density 
        p_mx = pm0(x, m_data, s_data).numpy() #data density

        
        p_ng = pn( g,m0, S).numpy() # noise (generator) density 
        p_mg = pm0(g, m_data, s_data).numpy() #data density

        grad_cte = 1/cte - p_mx/(cte*p_mx+p_nx) -  p_mg/(cte*p_mg+p_ng)
        grad_cte = np.sum(grad_cte)/batch_size
            
        
        cte = cte + 0.01*grad_cte
        
#         print("gradient   " , grad_cte )
#         print("cte" , cte)
        error_cte.append( (grad_cte) ) 
        if np.isnan(grad_cte):
            break
            
        opt.minimize(obj_f2, var_list=[a,b,c,d])
        

Instructions for updating:
`MultivariateNormalFullCovariance` is deprecated, use `MultivariateNormalTriL(loc=loc, scale_tril=tf.linalg.cholesky(covariance_matrix))` instead.


In [48]:
x1 = tf.Variable(10.0, dtype ="double")
x1, x2 = reset()
opt = tf.keras.optimizers.SGD(learning_rate=0.1)
for i in range(50):
	print ('y = {:.1f}, x1 = {:.1f}, x2 = {:.1f}'.format(fu(x1, x2).numpy(), x1.numpy(), x2.numpy()))
	opt.minimize(fu_minimzie, var_list=[x1])

y = 170.0, x1 = 10.0, x2 = 10.0
y = 141.3, x1 = 8.1, x2 = 10.0
y = 123.6, x1 = 6.6, x2 = 10.0
y = 112.7, x1 = 5.4, x2 = 10.0
y = 106.1, x1 = 4.4, x2 = 10.0
y = 102.2, x1 = 3.6, x2 = 10.0
y = 100.0, x1 = 3.0, x2 = 10.0
y = 98.7, x1 = 2.5, x2 = 10.0
y = 98.1, x1 = 2.1, x2 = 10.0
y = 97.8, x1 = 1.8, x2 = 10.0
y = 97.8, x1 = 1.5, x2 = 10.0
y = 97.8, x1 = 1.3, x2 = 10.0
y = 97.9, x1 = 1.2, x2 = 10.0
y = 98.0, x1 = 1.0, x2 = 10.0
y = 98.1, x1 = 0.9, x2 = 10.0
y = 98.2, x1 = 0.8, x2 = 10.0
y = 98.3, x1 = 0.8, x2 = 10.0
y = 98.4, x1 = 0.7, x2 = 10.0
y = 98.4, x1 = 0.7, x2 = 10.0
y = 98.5, x1 = 0.6, x2 = 10.0
y = 98.5, x1 = 0.6, x2 = 10.0
y = 98.6, x1 = 0.6, x2 = 10.0
y = 98.6, x1 = 0.6, x2 = 10.0
y = 98.6, x1 = 0.6, x2 = 10.0
y = 98.7, x1 = 0.5, x2 = 10.0
y = 98.7, x1 = 0.5, x2 = 10.0
y = 98.7, x1 = 0.5, x2 = 10.0
y = 98.7, x1 = 0.5, x2 = 10.0
y = 98.7, x1 = 0.5, x2 = 10.0
y = 98.7, x1 = 0.5, x2 = 10.0
y = 98.7, x1 = 0.5, x2 = 10.0
y = 98.7, x1 = 0.5, x2 = 10.0
y = 98.7, x1 = 0.5, x2 = 10.0
y 